### Arithmetic Operators

Давайте сначала рассмотрим простой пример использования простых `__add__`, `__sub__` и т. д.

Допустим, мы хотим реализовать класс `Vector`, который поддерживает различные арифметические операции. Мы не будем предполагать конкретное количество измерений — оно будет определяться тем, сколько аргументов передается методу `__init__`. Однако мы потребуем, чтобы аргументы были действительными числами.


In [1]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'

Теперь давайте поддержим сложение и вычитание векторов — они должны быть одинаковой размерности, в противном случае мы должны вызвать исключение `TypeError` (согласующееся с исключением, которое Python вызывает, например, при попытке сложить строку и целое число).

In [2]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)

Давайте попробуем наш класс и посмотрим, как все работает на данном этапе:

In [3]:
v1 = Vector(1, 2)
v2 = Vector(10, 10)
v3 = Vector(1, 2, 3, 4)

In [4]:
v1

Vector(1, 2)

In [5]:
v1 + v2

Vector(11, 12)

In [6]:
v2 + v1 

Vector(11, 12)

In [7]:
try:
    print(v1 + v3)
except TypeError as ex:
    print(ex)

unsupported operand type(s) for +: 'Vector' and 'Vector'


In [8]:
try:
    print(v1 + 100)
except TypeError as ex:
    print(ex)

unsupported operand type(s) for +: 'Vector' and 'int'


Теперь давайте добавим поддержку умножения на скалярное значение — например, умножение вектора на вещественное число (не другой вектор).

Для этого мы реализуем метод `__mul__`:

In [9]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if not isinstance(other, Real):
            return NotImplemented
        components = (other * x for x in self.components)
        return Vector(*components)

In [10]:
v1 = Vector(1, 2)

In [11]:
v1 * 10

__mul__ called...


Vector(10, 20)

Но что произойдет, если мы проделаем обратную операцию:

In [12]:
try:
    10 * v1
except TypeError as ex:
    print(ex)

unsupported operand type(s) for *: 'int' and 'Vector'


Здесь произошло следующее: Python сначала попытался вызвать операцию сложения для объекта `int`, используя `Vector` в качестве второго операнда. Целые числа, конечно, не поддерживают этот тип, поэтому Python попытался использовать наш класс `Vector`, но не `__mul__`, поскольку он вызывается, когда `Vector` является **левым** операндом. Вместо этого он ищет (и не находит) метод для использования, когда `Vector` является **правым** операндом.

Мы можем реализовать этот метод, используя `__rmul__`:

In [13]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if not isinstance(other, Real):
            return NotImplemented
        components = (other * x for x in self.components)
        return Vector(*components)
    
    def __rmul__(self, other):
        print('__rmul__ called...')
        # for us, multiplication is commutative, so we can leverage our existing __mul__ method
        return self * other

In [14]:
v1 = Vector(1, 2)

In [15]:
v1 * 10

__mul__ called...


Vector(10, 20)

In [16]:
10 * v1

__rmul__ called...
__mul__ called...


Vector(10, 20)

Теперь предположим, что мы хотим реализовать скалярное произведение двух векторов.

Если вы в этом не разбираетесь, просто быстро прочтите это: https://en.wikipedia.org/wiki/Dot_product

По сути, нам нужны векторы одинаковой размерности, и мы вычисляем сумму произведения компонентов (попарно) в каждом векторе.

Мы можем реализовать это, различая типы `Real` и ` Vector` в нашем методе `__mul__` - конечно, нам это не понадобится в методе `__rmul__`, потому что если мы реализуем умножение двух `Vector`, у нас всегда будет `Vector` в качестве левого операнда, поэтому `__mul__` будет вызван первым.

In [17]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if isinstance(other, Real):
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            # dot product
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented
    
    def __rmul__(self, other):
        print('__rmul__ called...')
        # for us, multiplication is commutative, so we can leverage our existing __mul__ method
        return self * other

In [18]:
v1 = Vector(1, 2)
v2 = Vector(3, 4)

In [19]:
v1 * v2

__mul__ called...


11

Мы также могли бы реализовать **перекрестное** произведение двух векторов (которое вернуло бы другой вектор).

Вычисления становятся немного сложнее, поэтому я не буду показывать вам эти подробности, но давайте посмотрим, как мы могли бы использовать оператор `@` для реализации этого:

In [20]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if isinstance(other, Real):
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            # dot product
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented
    
    def __rmul__(self, other):
        print('__rmul__ called...')
        # for us, multiplication is commutative, so we can leverage our existing __mul__ method
        return self * other
    
    def __matmul__(self, other):
        print('__matmul__ called...')

In [21]:
v1 = Vector(1, 2)
v2 = Vector(3, 4)

In [22]:
v1 * v2

__mul__ called...


11

In [23]:
v1 @ v2

__matmul__ called...


#### In-Place Operators

У нас также есть операторы на месте. Обычно операторы на месте пытаются **мутировать** объект слева от выражения:

In [24]:
l = [1, 2]

In [25]:
id(l)

140460867509512

In [26]:
l += [3]

In [27]:
id(l), l

(140460867509512, [1, 2, 3])

Как видите, список `l` mas мутировал (адрес памяти остался прежним). Это не тот же эффект, что и:

In [28]:
l = [1, 2]
print(id(l))

l = l + [3]
print(id(l), l)

140460867621128
140460867592136 [1, 2, 3]


Как видите, здесь мы получили **новый** объект списка.

Но in-place **не** *гарантирует* мутацию. Например, кортежи — это неизменяемые объекты:

In [29]:
t = (1, 2)
print(id(t))
t += (3, )
print(id(t), t)

140460615377736
140460867314816 (1, 2, 3)


Как видите, мы получили новый кортеж. То же самое происходит со строками, целыми числами, числами с плавающей точкой и т. д., которые также являются неизменяемыми типами.

Давайте вернемся к нашему классу `Vector` и реализуем сложение на месте, но мы реализуем его таким образом, чтобы не изменять Vector, а просто возвращать новый Vector, аналогично тому, что мы только что видели с кортежами:

In [30]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if isinstance(other, Real):
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            # dot product
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented
    
    def __rmul__(self, other):
        print('__rmul__ called...')
        # for us, multiplication is commutative, so we can leverage our existing __mul__ method
        return self * other
    
    def __iadd__(self, other):
        print('__radd__ called...')
        return self + other

In [31]:
v1 = Vector(1, 2)
v2 = Vector(10, 10)

print(id(v1))

v1 += v2

print(id(v1), v1)

140460867485200
__radd__ called...
140460867485312 Vector(11, 12)


Как вы видите, в итоге мы получаем новый объект `Vector`.

Теперь давайте изменим это так, чтобы мы фактически мутировали объект `Vector`:

In [32]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if isinstance(other, Real):
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            # dot product
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented
    
    def __rmul__(self, other):
        print('__rmul__ called...')
        # for us, multiplication is commutative, so we can leverage our existing __mul__ method
        return self * other
    
    def __iadd__(self, other):
        print('__radd__ called...')
        if self.validate_type_and_dimension(other):
            components = (x + y for x, y in zip(self.components, other.components))
            self._components = tuple(components)  # mutating our Vector object
            return self # don't forget to return the result of the operation!
        return NotImplemented
        

In [33]:
v1 = Vector(1, 2)
v2 = Vector(10, 20)

print(id(v1))

v1 += v2

print(id(v1), v1)

140460867518080
__radd__ called...
140460867518080 Vector(11, 22)


Как видите, мы **мутировали** объект `v1`.

Давайте также реализуем унарный минус в нашем классе `Vector`. В этом случае мы просто хотим вернуть новый `Vector` с каждым инвертированным компонентом:

In [34]:
from numbers import Real

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if isinstance(other, Real):
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            # dot product
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented
    
    def __rmul__(self, other):
        print('__rmul__ called...')
        # for us, multiplication is commutative, so we can leverage our existing __mul__ method
        return self * other
    
    def __iadd__(self, other):
        print('__radd__ called...')
        if self.validate_type_and_dimension(other):
            components = (x + y for x, y in zip(self.components, other.components))
            self._components = tuple(components)  # mutating our Vector object
            return self # don't forget to return the result of the operation!
        return NotImplemented
        
    def __neg__(self):
        print('__neg__ called...')
        components = (-x for x in self.components)
        return Vector(*components)

In [35]:
v1 = Vector(1, 2)
-v1

__neg__ called...


Vector(-1, -2)

Поэтому мы можем использовать его в арифметических операциях, таких как:

In [36]:
v2 = Vector(10, 10)

v2 + -v1

__neg__ called...


Vector(9, 8)

Наконец, реализуем функцию `abs` для нашего вектора. Сейчас это не будет работать:

In [37]:
try:
    abs(v1)
except TypeError as ex:
    print(ex)

bad operand type for abs(): 'Vector'


Но мы можем это исправить:

In [38]:
from numbers import Real
from math import sqrt

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if isinstance(other, Real):
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            # dot product
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented
    
    def __rmul__(self, other):
        print('__rmul__ called...')
        # for us, multiplication is commutative, so we can leverage our existing __mul__ method
        return self * other
    
    def __iadd__(self, other):
        print('__radd__ called...')
        if self.validate_type_and_dimension(other):
            components = (x + y for x, y in zip(self.components, other.components))
            self._components = tuple(components)  # mutating our Vector object
            return self # don't forget to return the result of the operation!
        return NotImplemented
        
    def __neg__(self):
        print('__neg__ called...')
        components = (-x for x in self.components)
        return Vector(*components)
    
    def __abs__(self):
        print('__abs__ called...')
        return sqrt(sum(x ** 2 for x in self.components))

In [39]:
v1 = Vector(1, 1)

In [40]:
abs(v1)

__abs__ called...


1.4142135623730951

#### Other Uses

Конечно, эти арифметические операторы не ограничиваются работой с числами. Мы видели, как они работают также со строками, например, или даже списками.

Мы также можем использовать их в наших пользовательских классах разными способами, где мы хотим реализовать и придать особое значение этим операторам.

Например, у нас может быть класс `Family`, который содержит вместе:
- объекты матери и отца `Person`
- список детей `Person`

Мы хотим сделать так, чтобы мы могли добавлять детей, просто используя сложение на месте.

In [41]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f"Person('{self.name}')"

In [42]:
p1 = Person('John')

In [43]:
class Family:
    def __init__(self, mother, father):
        self.mother = mother
        self.father = father
        self.children = []
        
    def __iadd__(self, other):
        self.children.append(other)
        return self
    

In [44]:
f = Family(Person('Mary'), Person('John'))
print(id(f))

140460867516680


In [45]:
f += Person('Eric')
print(id(f))
print(f.children)

140460867516680
[Person('Eric')]


In [46]:
f += Person('Michael')
print(id(f))
print(f.children)

140460867516680
[Person('Eric'), Person('Michael')]


Поэтому не ограничивайтесь использованием этих операторов только для числовых случаев.